In [1]:
# Intro

# Preprocessing

# Training

# Inference


# Intro


The surrogate's goal is to emulate the [wflow hydrological model](https://deltares.github.io/Wflow.jl/stable/) outputs,in particular:

- Soil moisture
- Actual evapotranspiration
- River discharge

This notebook shows the 


# Preprocessing

Wflow runs on a gridded domain, discretized in time and space. The temporal resolution is 1 day and the spatial resolution is 1 km. The data temporal range that can be used for training and test is from 2007 to 2020.

The wflow input/output data structure has 4 dimensions: lat, lon, time, feature.  

The LSTM models works with a different structure: gridcell, time, feature. Therefore there is a need for reshaping the data back and forth for training and the visualization of the inference results on a map.
